## Function Declaration (cont'd)


In [1]:
class FaceOrientationCounter:
    def __init__(self):
        # Initialize counters for each orientation
        self.counters = {90: 0, 270: 0}
        
    def update(self, angle, faces_detected):
        # Increase the counter for the current angle if faces are detected
        if faces_detected > 0:
            self.counters[angle] += faces_detected
            
        # Decrease the counters slightly each time to favor recent observations
        for ang in self.counters:
            if ang != angle:
                self.counters[ang] *= 0.9  # Decay factor for 'forgetting' older observations
                
    def get_preferred_orientation(self):
        # Determine the orientation with the highest counter value
        preferred_angle = max(self.counters, key=self.counters.get)
        return preferred_angle

def detect_faces_in_rotations_2(frame, face_cascade, orientation_counter=None):
    # Define the rotation angles
    rotation_angles = [90, 270]
    detected_faces = []

    original_w, original_h = frame.shape[1], frame.shape[0]
    most_faces_detected = 0
    best_angle = 0

    for angle in rotation_angles:
        # Rotate the image
        rotated_frame, M = rotate_image(frame, angle)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(rotated_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        if len(faces) > most_faces_detected:
            # If more faces are detected, update the counter and the best angle
            most_faces_detected = len(faces)
            best_angle = angle
            detected_faces = [(x, y, w, h) for (x, y, w, h) in faces]
            
    # Update orientation counter if it's provided
    if orientation_counter is not None:
        orientation_counter.update(best_angle, most_faces_detected)
        best_angle = orientation_counter.get_preferred_orientation()

    # Adjust detected faces based on the best angle
    adjusted_faces = []
    if most_faces_detected > 0:
        for (x, y, w, h) in detected_faces:
            # Calculate the inverse rotation for the detected face coordinates based on the best angle
            rotated_frame, M = rotate_image(frame, best_angle)
            inv_M = cv2.invertAffineTransform(M)
            original_coords = np.array([[x + w/2, y + h/2]], dtype=np.float32)
            original_coords = np.array([original_coords])
            # Transform back the coordinates of the center of the face
            transformed_coords = cv2.transform(original_coords, inv_M)
            ox, oy = transformed_coords[0][0]
            ox, oy = int(ox - w/2), int(oy - h/2)
            
            # Store the corrected face box
            adjusted_faces.append((ox, oy, w, h))

    return adjusted_faces


In [2]:
def timer(frames_processed, start_time, fps, total_frames):
    frames_processed += 1

    # Calculate the average processing time per frame up to this point
    elapsed_time = (time.time() - start_time)
    average_time_per_frame = elapsed_time / frames_processed

    # Calculate the estimated remaining time
    remaining_frames = total_frames - frames_processed
    estimated_remaining_time = remaining_frames * average_time_per_frame
    print(f"Total frames: {total_frames}, Remaining frames: {remaining_frames}, Est. remaining time: {estimated_remaining_time:.1f} s, Total elapsed time: {elapsed_time:.1f}, FPS: {1/average_time_per_frame:.2f}")
        
    return frames_processed

In [3]:
def draw_safe_zone(image, safe_zone):
    """
    Draw a green rectangle around the safe zone.
    """
    x, y, w, h = safe_zone
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)
    return image

In [4]:
def add_safe_zone_to_video(input_video_path, output_video_path):
    
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    
    # Obtain video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the codec and create a VideoWriter object to write the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' if you face issues with 'mp4v'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    frames_processed = 0
    start_time = time.time()
    
    face_cascade = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')
    
    # Initialize the counter object outside of your frame processing loop
    orientation_counter = FaceOrientationCounter()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Add the green frame into the frame
        edited_frame = frame_checker(face_cascade, frame, orientation_counter)
        
        # Write the frame with the safe zone to the output video
        out.write(edited_frame)
        
        # Frame counter/ timer (optional)
        frames_processed = timer(frames_processed, start_time, fps, total_frames)
        
        # Break the loop if 'q' is pressed (for real-time viewing)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

## Function Caller

In [5]:
# # Example usage
# input_video_path = 'samples/video_test_4_trimmed.mp4'
# output_video_path = input_video_path[:-4] + '_output.mp4'
# add_safe_zone_to_video(input_video_path, output_video_path)